# Đồ án môn học Lập trình song song ứng dụng

# Phát hiện khuôn mặt bằng Cascaded-Classifiers

## Giáo viên: Trần Trung Kiên

## Sinh viên:
- 1712628 - Huỳnh Hiếu Nhân
- 1712657 - Chung Kiệt Phong
- 1712869 - Đào Duy Tuấn

---

# 1. Mô tả ứng dụng

- Nhóm em chọn ứng dụng Phát hiện khuôn mặt bằng Cascaded-Classifiers
	- Input: một tấm ảnh RGB
	- Output: tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)
	- Ý nghĩa của ứng dụng trong thực tế:
		- Hỗ trợ tính năng autofocus trên máy ảnh
		- Là bước quan trọng trong các hệ thống sử dụng nhận diện khuôn mặt: giám sát, an ninh, cơ sở dữ liệu, chăm sóc khách hàng ...
		- Ứng dụng để tự động đề xuất gắn thẻ cho ảnh trên mạng xã hội
- Lý do cần tăng tốc ứng dụng:
	- Có nhiều ý nghĩa thực tế
	- Ứng dụng sẽ chạy chậm nếu cài đặt tuần tự
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao, video, streaming video trong thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Các khung cửa sổ để phát hiện khuôn mặt ở mọi vị trí trong ảnh và các kích thước khác nhau của khung cửa số xử lý độc lập với nhau
  - Tính tuần tự của các giai đoạn kiểm tra có thách thức lớn nhưng tốc độ xử lý được gia tăng đáng kể nếu có thể song song hóa phần này

---

# 2. Cài đặt tuần tự

## 2.1. Thiết kế

Khái quát về Haar Cascaded-Classifiers:
- Brute-force
    
    ![brute-force.gif](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/brute-force.gif)
- Haar-like feature

    ![haar-like-feature.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/haar-like-feature.png)
    
    Một số Feature cơ bản:

    ![haar-features.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/haar-features.png)

- Summed Area Table
- Cascaded-classifiers

    ![cascaded-classifiers.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/cascaded-classifiers.png)



Các bước triển khai:
- Input: một tấm ảnh RGB, mô hình Haar Cascade đã train sẵn

- Chuyển ảnh đầu vào sang ảnh xám.

- Đọc mô hình Haar Cascade: gồm nhiều Classifier, mỗi Classifier có một threshold và nhiều Tree, mỗi Tree gồm các node: node trong chứa Feature và threshold; node lá chứa giá trị trả về. Mỗi Feature được định nghĩa bởi hai hoặc ba Rectangle.

    Bên trong một Classifier:

    <img src=https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/classifier.jpg alt="classifier.jpg" width="700"/>


- Tính Summed Area Table (SAT) là một ma trận có kích thước như ảnh đầu vào. Giá trị tại vị trí $(x, y)$ của bảng SAT được tính bằng tổng độ xám của các pixel trong hình chữ nhật có tọa độ 2 góc đối là $(0, 0)$ và $(x, y)$.

    Cách tính bảng SAT và sử dụng để tính nhanh Feature:
    ![summed-area-table.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/summed-area-table.png)

- Trượt khung cửa sổ có kích thước ban đầu là 24x24 trên ảnh xám. Vùng ảnh trong khung cửa sổ được đưa vào Cascaded-Classifiers. Các Tree trong một Classifier sẽ tính Feature mà nó giữ để quyết định giá trị trả về. Classifier lấy tổng giá trị của các Tree, so sánh với threshold để quyết định đẩy vùng ảnh đang xét sang Classifier sau hay loại bỏ. Ảnh cứ thế được lan truyền qua các Classifier cho tới khi bị loại bỏ hoặc ra khỏi Classifier cuối cùng thì vùng ảnh trong khung cửa sổ được ghi nhận lại. Lặp lại quá trình trên với khung cửa sổ có kích thước lớn hơn cho tới tới khi kích thước khung cửa sổ vượt quá kích thước ảnh.

    Minh hoạ:
    
    ![sliding-dectection-window.gif](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/sliding-dectection-window.gif)

- Gom nhóm các khung cửa sổ phát hiện cùng một khuôn mặt: hai khung cửa sổ có khoảng cách giữa các cạnh tương ứng đều nhỏ hơn một hằng số $\epsilon$ chỉ định trước thì thuộc cùng một nhóm và ngược lại. Tiếp theo:
	- Loại bỏ các nhóm có số khung cửa sổ thấp hơn ngưỡng chỉ định trước (giảm false positive).
	- Ở mỗi nhóm được giữ lại, tính trung bình tọa độ các khung cửa sổ trong nhóm để gộp các khung thành một bounding box duy nhất và vẽ lên ảnh kết quả.

- Output: tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)

---